In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

In [ ]:
# Инициализируем модель и токенизатор

model_name = "cointegrated/rubert-tiny"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

In [ ]:
# Загружаем данные

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/valid.csv')

print(train.shape)
print(test.shape)

In [ ]:
# Проверяем наличие пустых строк

print(f'Train:\n{train.isna().sum()}')
print()
print(f'Test: \n{test.isna().sum()}')

In [ ]:
# Создаем словарь, в котором число соответсвует сентименту сообщения

sentiment_dict = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

In [ ]:
# Преобразуем данные в подходящий для модели формат

train_set = Dataset.from_pandas(train)
test_set = Dataset.from_pandas(test)

In [ ]:
# Создадим функцию для токенизации текстов, токенизируем данные

def tokenize(batch):
    return tokenizer(batch["text"], max_length=256, padding='max_length', truncation=True)

train = train_set.map(tokenize, batched=True)
test = test_set.map(tokenize, batched=True)

In [ ]:
# Переименуем целевую переменную
train = train.rename_column("sentiment", "labels")
test = test.rename_column("sentiment", "labels")

# Удалим ненужные признаки
train = train.remove_columns(["Unnamed: 0", "text"])
test = test.remove_columns(["Unnamed: 0", "text"])

In [ ]:
train

In [ ]:
# Функция для отображения метрик качества

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


# Так же добавим в конфигурацию словарь соответсвия сентиментов для человекочитаемого отображения

model.config.id2label = sentiment_dict
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [ ]:
args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

GitHub не отображает виджеты, точность модели после обучения 0.77

In [ ]:
model.save_pretrained("model/")
tokenizer.save_pretrained("model/")

Вы можете скачать первый файл с таким же названием из репозитория, чтобы увидеть виджеты и результаты выполнения ячеек. Локально jupiter все покажет. 